<a href="https://colab.research.google.com/github/Albertooo18/Proyecto_final/blob/main/Google_colabs/clasificacion_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Crear las carpetas para subir las imagenes
!mkdir apples
!mkdir bananas

In [ ]:
#Entrar en cada carpeta y descomprimir el archivo zip
%cd apples
!unzip apples.zip
%cd ..

In [ ]:
%cd bananas
!unzip bananas.zip
%cd ..

In [ ]:
!rm -rf /content/apples/apples.zip
!rm -rf /content/bananas/bananas.zip

In [ ]:
#Mostrar cuantas imagenes tengo de cada categoria
!ls /content/apples/apples | wc -l
!ls /content/bananas/bananas | wc -l

In [ ]:
!mkdir dataset
!mkdir dataset/apple
!mkdir dataset/banana

In [ ]:
import shutil
carpeta_fuente = '/content/apples/apples'
carpeta_destino = '/content/dataset/apple'

imagenes = os.listdir(carpeta_fuente)

for i, nombreimg in enumerate(imagenes):
  if i < 190:
    #Copia de la carpeta fuente a la destino
    shutil.copy(carpeta_fuente + '/' + nombreimg, carpeta_destino + '/' + nombreimg)

In [ ]:
carpeta_fuente = '/content/bananas/bananas'
carpeta_destino = '/content/dataset/banana'

imagenes = os.listdir(carpeta_fuente)

for i, nombreimg in enumerate(imagenes):
  if i < 190:
    #Copia de la carpeta fuente a la destino
    shutil.copy(carpeta_fuente + '/' + nombreimg, carpeta_destino + '/' + nombreimg)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
""" carpeta_entrenamiento = os.path.join('/content/drive/MyDrive/clasificacion_de_imagenes/clasificacion_2.0/Fruits_Dataset_1.1/Train_1.1')
carpeta_validacion = os.path.join('/content/drive/MyDrive/clasificacion_de_imagenes/clasificacion_2.0/Fruits_Dataset_1.1/Test_1.1')  """


In [ ]:
""" carp_entren_apple = os.path.join('/content/drive/MyDrive/clasificacion_de_imagenes/clasificacion_2.0/Fruits_Dataset_1.1/Train_1.1/apple_1.1')  # imagenes de manzanas para pruebas
carpeta_entren_banana = os.path.join('/content/drive/MyDrive/clasificacion_de_imagenes/clasificacion_2.0/Fruits_Dataset_1.1/Train_1.1/banana_1.1')  # imagenes de bananas para pruebas
carpeta_val_apple = os.path.join('/content/drive/MyDrive/clasificacion_de_imagenes/clasificacion_2.0/Fruits_Dataset_1.1/Test_1.1/apple_1.1')  # imagenes de manzanas para validacion
carpeta_val_banana = os.path.join('/content/drive/MyDrive/clasificacion_de_imagenes/clasificacion_2.0/Fruits_Dataset_1.1/Test_1.1/banana_1.1')  # imagenes de bananas para validacion """

In [ ]:
num_apple_entren = len(os.listdir(carp_entren_apple))
num_banana_entren = len(os.listdir(carpeta_entren_banana))
num_apple_val = len(os.listdir(carpeta_val_apple))
num_banana_val = len(os.listdir(carpeta_val_banana))
total_entrenamiento = num_apple_entren + num_banana_entren
total_val = num_apple_val + num_banana_val

TAMANO_LOTE = 100
TAMANO_IMG = 150

In [ ]:
#Aumento de datos
print("Realizando aumento de datos")
image_gen_entrenamiento = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')


Realizando aumento de datos


In [ ]:
#Generacion de datos de entrenamiento FTW
data_gen_entrenamiento = image_gen_entrenamiento.flow_from_directory(batch_size=TAMANO_LOTE,
                                                     directory=carpeta_entrenamiento,
                                                     shuffle=True,
                                                     target_size=(TAMANO_IMG,TAMANO_IMG),
                                                     class_mode='binary')

#Generacion de datos de validacion
image_gen_val = ImageDataGenerator(rescale=1./255)

data_gen_validacion = image_gen_val.flow_from_directory(batch_size=TAMANO_LOTE,
                                                 directory=carpeta_validacion,
                                                 target_size=(TAMANO_IMG, TAMANO_IMG),
                                                 class_mode='binary')

Found 203 images belonging to 2 classes.
Found 187 images belonging to 2 classes.


In [ ]:
#Modelo!
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2)
])

In [ ]:
#Compilación
modelo.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#Entrenamiento
print("Entrenando modelo...");
epocas=60
history = modelo.fit_generator(
    data_gen_entrenamiento,
    steps_per_epoch=int(np.ceil(total_entrenamiento / float(TAMANO_LOTE))),
    epochs=epocas,
    validation_data=data_gen_validacion,
    validation_steps=int(np.ceil(total_val / float(TAMANO_LOTE)))
)

print("Modelo entrenado!");

Entrenando modelo...


<ipython-input-31-cac65dffc3e0>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = modelo.fit_generator(


Epoch 1/60
3/3 [==============================] - 14s 1s/step - loss: 0.6518 - accuracy: 0.4877 - val_loss: 0.9420 - val_accuracy: 0.5722
Epoch 2/60
3/3 [==============================] - 3s 1s/step - loss: 0.6583 - accuracy: 0.5369 - val_loss: 0.6313 - val_accuracy: 0.4706
Epoch 3/60
3/3 [==============================] - 3s 1s/step - loss: 0.6907 - accuracy: 0.5419 - val_loss: 0.6553 - val_accuracy: 0.4545
Epoch 4/60
3/3 [==============================] - 3s 1s/step - loss: 0.5549 - accuracy: 0.7291 - val_loss: 0.5379 - val_accuracy: 0.8235
Epoch 5/60
3/3 [==============================] - 3s 943ms/step - loss: 0.5462 - accuracy: 0.7389 - val_loss: 0.5623 - val_accuracy: 0.6738
Epoch 6/60
3/3 [==============================] - 3s 894ms/step - loss: 0.5190 - accuracy: 0.6749 - val_loss: 0.4155 - val_accuracy: 0.8021
Epoch 7/60
3/3 [==============================] - 3s 1s/step - loss: 0.3258 - accuracy: 0.9015 - val_loss: 0.2515 - val_accuracy: 0.8877
Epoch 8/60
3/3 [==================

In [ ]:
modelo.save('apple-banana.h5')

In [ ]:
!pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.4/197.4 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 KB 247.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 KB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
!mkdir carpeta_salida

In [ ]:
!tensorflowjs_converter --input_format keras apple-banana.h5 carpeta_salida

2023-01-15 07:30:52.111735: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-01-15 07:30:52.111977: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-01-15 07:30:52.112016: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [ ]:
!ls carpeta_salida

group1-shard1of4.bin  group1-shard3of4.bin  model.json
group1-shard2of4.bin  group1-shard4of4.bin
